In [1]:
def plotstuff(cell, t, data, electrodes, dim, mag, real_morph, scale_mult):
    '''plotting'''
    fig = plt.figure(dpi=160)
    
    ax1 = fig.add_axes([0.05, 0.1, 0.55, 0.9], frameon=False)
    cax = fig.add_axes([0.05, 0.115, 0.55, 0.015])
    
    ax1.plot(electrodes.y, electrodes.z, '.', marker='o', markersize=3, color='k', zorder=0)
    
    #normalize to min peak
    LFPmin = data.min(axis=1)
    #print("LFPmin: ", LFPmin)
    mins = np.ones((np.shape(data)[0], 1))
    mins = mins * data.min()
    mins = np.squeeze(mins)
    #LFPmin = electrode.data.min()
    LFPnorm = -(data.T / mins).T
    #LFPnorm = electrode.data / LFPmin
    
    #print(LFPmin.min)
    
#     all_B_y = np.squeeze(all_B[:, 1, :])*1e12
#     all_B_y_min = all_B_y.min(axis=1)
#     all_B_y_norm = -(all_B_y.T / all_B_y_min).T
    
#     LFPmin = all_B_y_min
#     LFPnorm = all_B_y_norm
    
    i = 0
    zips = []
    width = int(ax1.bbox.width/8*0.93)
    mult = width/t[-1]
    print(mult)
    for x in LFPnorm:
        zips.append(list(zip(t*6 + electrodes.y[i] + 2,
                        x*scale_mult + electrodes.z[i])))
        i += 1
    
    line_segments = LineCollection(zips,
                                    linewidths = (1.75),
                                    linestyles = 'solid',
                                    cmap='nipy_spectral',
                                    zorder=1,
                                    rasterized=False)
    line_segments.set_array(np.log10(-LFPmin))
    ax1.add_collection(line_segments)
    
    axcb = fig.colorbar(line_segments, cax=cax, orientation='horizontal')
    axcb.outline.set_visible(False)
    xticklabels = np.array([-0.1  , -0.05 , -0.02 , -0.01 , -0.005, -0.002])
    if mag == True:
        xticklabels = np.array([-50, -20, -10, -2, -1, -0.5, -0.1, -0.05, -0.01])
    xticks = np.log10(-xticklabels)
    axcb.set_ticks(xticks)
    axcb.set_ticklabels(np.round(-10**xticks, decimals=3))
    axcb.set_label('spike amplitude (mV)', va='center')
    if mag == True:
        axcb.set_label('spike amplitude (pT)', va='center')
    
    #ax1.plot([-150, 38], [100, 100], color='k', lw = 1)
    #ax1.text(-150, 102, '10 ms')
    
    #ax1.plot([60, 80], [100, 100], color='k', lw = 1)
    #ax1.text(150, 102, '20 $\mu$m')
    
    ax1.set_xticks([])
    ax1.set_yticks([])
    
    axis = ax1.axis(ax1.axis('equal'))
    ax1.set_xlim(axis[0]*1.02, axis[1]*1.02)
    
    # plot morphology
    if cell != None:
        pt3d_polygons = []
        if real_morph == False:
            for sec in cell.allseclist:
                x = []
                y = []
                d = []
                for i in range(sec.n3d()):
                    x.append(sec.x3d(i))
                    y.append(sec.y3d(i))
                    d.append(sec.diam3d(i))
                polys = create_polygon2(x, y, d)
                pt3d_polygons.append(polys)
        else:
            print("Real morph")
            pt3d_polygons = cell.get_pt3d_polygons(projection=('y', 'z'))
    
        zips = []
        for x, y in pt3d_polygons:
            zips.append(list(zip(x, y)))
        from matplotlib.collections import PolyCollection
        polycol = PolyCollection(zips, edgecolors='none', facecolors='gray', zorder=-1, rasterized=False)
        ax1.add_collection(polycol)

        ax1.text(-0.05, 0.95, 'a', horizontalalignment='center', verticalalignment='center', fontsize=16, fontweight='demibold', transform=ax1.transAxes)

    return fig

In [2]:
def plotstuff2(cell, t, data, electrodes, dim, mag, real_morph, scale_mult, time_scale, plot_full_cell = False):
    '''plotting'''
    fig = plt.figure(dpi=160)
    
    ax1 = fig.add_axes([0.05, 0.1, 0.55, 0.9], frameon=False)
    cax = fig.add_axes([0.05, 0.115, 0.55, 0.015])
    
    # Plot electrode positions
    ax1.plot(electrodes.y, electrodes.z, '.', marker='o', markersize=3, color='k', zorder=0)
    if plot_full_cell:
        max_y, min_y, max_z, min_z = get_cell_extents(cell)
        ax1.plot([max_y, min_y], [max_z, min_z], '.', marker='o', markersize=1, color="white", zorder=1)
    
    #normalize to max peak
    maxs = np.ones((np.shape(data)[0], 1))
    maxs = maxs * np.abs(data).max()
    maxs = np.squeeze(maxs)
    LFPnorm = (data.T / maxs).T
        
    # Calculate time scale to adjust width of traces to fit perfectly between plotted electrodes
    
    # Find distance along y axis between electrodes
    start_y = electrodes.y[0]
    delta_y = 0
    for i in range(1, len(electrodes.y)):
        if electrodes.y[i] != start_y:
            delta_y = electrodes.y[i] - start_y
            break
    
    # Adjust time_scale to fit within 80% of distance between electrodes
#     time_scale = 10
    if delta_y > 0 and time_scale == 0:
        delta_t = t[-1]-t[0]
        time_scale = delta_y/delta_t*0.8
        
    zips = []
    for i in range(len(LFPnorm)):
        zips.append(list(zip((t-t[0])*time_scale + electrodes.y[i], LFPnorm[i]*scale_mult+electrodes.z[i])))
        #ax1.plot((t-t[0])*time_scale + electrodes.y[i], LFPnorm[i]*scale_mult+electrodes.z[i])
    #cmap used to be nipy-spectral
    line_segments = LineCollection(zips,
                                   linewidths = (1.75),
                                   linestyles = 'solid',
                                   cmap='jet',
                                   zorder=1,
                                   rasterized=False)
    # set_array enables color scaling - done according to log of peak value for each trace
    scale_factor = 1 # Scales from mV for electrical recordings
    if mag == True:
        scale_factor = 1e12 # Scales from pT for magnetic recordings
    max_vals = np.abs(data*scale_factor).max(axis=1)
    line_segments.set_array(np.log10(max_vals))
    ax1.add_collection(line_segments)
    
    # add color bar
    axcb = fig.colorbar(line_segments, cax=cax, orientation='horizontal')
    axcb.outline.set_visible(False)

    # set color bar ticks to correspond to actual peak values (invert the logarithm)
    xticks = axcb.get_ticks()
    xticks = np.append(xticks, np.log10(np.abs(data).max()*scale_factor))
    axcb.set_ticks(xticks)
    axcb.set_ticklabels(np.round(10**xticks, decimals=2))
    axcb.set_label('spike amplitude (mV)', va='center')
    if mag == True:
        axcb.set_label('spike amplitude (pT)', va='center')
    
    ax1.set_xticks([])
    ax1.set_yticks([])
    
    axis = ax1.axis(ax1.axis('equal'))
    ax1.set_xlim(axis[0]*1.02, axis[1]*1.02)
    
    # plot morphology
    if cell != None:
        pt3d_polygons = []
        if real_morph == False:
            for sec in cell.allseclist:
                x = []
                y = []
                d = []
                for i in range(sec.n3d()):
                    x.append(sec.x3d(i))
                    y.append(sec.y3d(i))
                    d.append(sec.diam3d(i))
                polys = create_polygon2(x, y, d)
                pt3d_polygons.append(polys)
        else:
            print("Real morph")
            pt3d_polygons = cell.get_pt3d_polygons(projection=('y', 'z'))
    
        zips = []
        for x, y in pt3d_polygons:
            zips.append(list(zip(x, y)))
        from matplotlib.collections import PolyCollection
        polycol = PolyCollection(zips, edgecolors='none', facecolors='gray', zorder=-1, rasterized=False)
        ax1.add_collection(polycol)

#         ax1.text(-0.05, 0.95, 'a',
#                  horizontalalignment='center',
#                  verticalalignment='center',
#                  fontsize=16, fontweight='demibold',
#                  transform=ax1.transAxes)

    return fig

In [4]:
def plotcell(cell, X, Y, xlim, ylim, projection):
    fig = plt.figure(dpi=160)
    
    ax1 = fig.add_axes([0.05, 0.1, 0.55, 0.9], frameon=False)
    ax1.plot(X, Y, '.', marker='o', markersize=3, color='k', zorder=0)
    ax1.plot([xlim, xlim, -xlim, -xlim], [ylim, -ylim, ylim, -ylim], '.', marker='o', markersize=1, color='blue')
    pt3d_polygons = cell.get_pt3d_polygons(projection=projection)
    
    zips = []
    for x, y in pt3d_polygons:
        zips.append(list(zip(x, y)))
    from matplotlib.collections import PolyCollection
    polycol = PolyCollection(zips, edgecolors='none',
                             facecolors='gray', zorder=-1, rasterized=False)
    ax1.add_collection(polycol)
    return fig

In [1]:
def plotcell_3planes(cell, electrodes, lim, plot_electrodes=False):
    fig = plt.figure(figsize = (36, 12), dpi=160)
    
    projections = [('y', 'z'), ('x', 'y'), ('x', 'z')]
    if plot_electrodes:
        elec_pairs = [(electrodes.y, electrodes.z), (electrodes.x, electrodes.y), (electrodes.x, electrodes.z)]
    
    for i in range(len(projections)):
        projection = projections[i]
        if plot_electrodes:
            X, Y = elec_pairs[i]
        
        # Plot electrodes as well as points at the "limits" of the plot to ensure entire space is plotted
        ax = fig.add_axes([0.3*i, 0, 0.25, 1], frameon=False)
        if plot_electrodes == True:
            ax.plot(X, Y, '.', marker='o', markersize=3, color='k', zorder=0)
        ax.plot([lim, lim, -lim, -lim], [lim, -lim, lim, -lim], '.', marker='o', markersize=1, color='blue')
        
        # Plot neuron itself
        pt3d_polygons = cell.get_pt3d_polygons(projection=projection)
        zips = []
        for x, y in pt3d_polygons:
            zips.append(list(zip(x, y)))
        from matplotlib.collections import PolyCollection
        polycol = PolyCollection(zips, edgecolors='none',
                                 facecolors='gray', zorder=-1, rasterized=False)
        ax.add_collection(polycol)
        ax.set_title(projection[0] + "-" + projection[1])
    
    return fig

def plotcell_3planes2(cell, x, y, z, lim, plot_electrodes=False):
    fig = plt.figure(figsize = (36, 12), dpi=160)
    
    projections = [('y', 'z'), ('x', 'y'), ('x', 'z')]
    elec_pairs = [(y, z), (x, y), (x, z)]
    
    for i in range(len(projections)):
        projection = projections[i]
        X, Y = elec_pairs[i]
        
        # Plot electrodes as well as points at the "limits" of the plot to ensure entire space is plotted
        ax = fig.add_axes([0.3*i, 0, 0.25, 1], frameon=False)
        if plot_electrodes == True:
            ax.plot(X, Y, '.', marker='o', markersize=3, color='k', zorder=0)
        ax.plot([lim, lim, -lim, -lim], [lim, -lim, lim, -lim], '.', marker='o', markersize=1, color='blue')
        
        # Plot neuron itself
        pt3d_polygons = cell.get_pt3d_polygons(projection=projection)
        zips = []
        for x, y in pt3d_polygons:
            zips.append(list(zip(x, y)))
        from matplotlib.collections import PolyCollection
        polycol = PolyCollection(zips, edgecolors='none',
                                 facecolors='gray', zorder=-1, rasterized=False)
        ax.add_collection(polycol)
        ax.set_title(projection[0] + "-" + projection[1])
    
    return fig

def plotcell_1plane_morph(cell, xlim, ylim):
    fig = plt.figure(figsize = (36, 12), dpi=160)
    
    projections = [('y', 'z')]
    
    for i in range(len(projections)):
        projection = projections[i]
        
        # Plot electrodes as well as points at the "limits" of the plot to ensure entire space is plotted
        ax = fig.add_axes([0.3*i, 0, 0.25, 1], frameon=False)
        ax.plot([xlim, xlim, -xlim, -xlim], [ylim, -ylim, ylim, -ylim], '.', marker='o', markersize=1, color='blue')
        
        # Plot neuron itself
        pt3d_polygons = cell.get_pt3d_polygons(projection=projection)
        zips = []
        for x, y in pt3d_polygons:
            zips.append(list(zip(x, y)))
        from matplotlib.collections import PolyCollection
        polycol = PolyCollection(zips, edgecolors='none',
                                 facecolors='black', zorder=-1, rasterized=False)
        ax.add_collection(polycol)
        ax.set_title(projection[0] + "-" + projection[1])
    
    return fig, ax

In [3]:
def calc_scaling(X, Y, electrode, all_B):
    num_elecs = np.shape(all_B)[0]
    elec_coords = Y[0]
    if len(elec_coords) != num_elecs:
        elec_coords = X[:, 0]

    mag_y = np.zeros((num_elecs, 1))
    mag_z = np.zeros((num_elecs, 1))
    for i in range(num_elecs):
        mag_y[i] = np.abs(all_B[i, 1]).max()
        mag_z[i] = np.abs(all_B[i, 2]).max()

    num_elecs = np.shape(all_B)[0]
    elec_val = np.zeros((num_elecs, 1))
    for i in range(num_elecs):
        elec_val[i] = np.abs(electrode.data)[i].max()

    #dist = np.sqrt((Y[0]*1e-6)**2 + (Z[0,0,0]*1e-6)**2)
    #mu = 4*np.pi*1e-7
    #biot = mu*6e-9/(2*np.pi*dist)

    mag_y = mag_y/np.max(mag_y)
    mag_z = mag_z/np.max(mag_z)
    elec_val = elec_val/np.max(elec_val)
    #biot = biot/np.max(biot)
    
    return elec_coords, mag_y, mag_z, elec_val

def calc_width(trace, global_target=0):
    target = np.abs(trace).max()*0.25
    if np.abs(trace).max() != trace.max():
        target = -1*target
        
    if global_target != 0:
        target = global_target
    
    #print("Target: ", target)
        
    idx = np.argwhere(np.diff(np.sign(trace - target))).flatten()
    x1 = 0
    x2 = 0
    if len(idx) >= 2:
        x11 = idx[0]
        x12 = x11+1

        y11 = trace[x11]
        y12 = trace[x12]

        x1 = (target-y11)/(y12-y11)*(x12-x11)+x11

        x21 = idx[1]
        x22 = x21+1

        y21 = trace[x21]
        y22 = trace[x22]

        x2 = (target-y21)/(y22-y21)*(x22-x21)+x21
        
    return x1, x2

def calc_width_scaling(X, Y, electrode_data, all_B, global_target=[0, 0, 0]):
    num_elecs = len(X[0])
    elec_coords = Y[0]
    if len(elec_coords) != num_elecs:
        elec_coords = X[:, 0]

    mag_y_widths = np.zeros((num_elecs, 1))
    mag_z_widths = np.zeros((num_elecs, 1))
    elec_widths = np.zeros((num_elecs, 1))
    for i in range(num_elecs):
        mag_y_vals = calc_width(all_B[i, 1], global_target=global_target[0])
        mag_z_vals = calc_width(all_B[i, 2], global_target=global_target[1])
        elec_vals = calc_width(electrode_data[i], global_target=global_target[2])
        
        mag_y_widths[i] = mag_y_vals[1]-mag_y_vals[0]
        mag_z_widths[i] = mag_z_vals[1]-mag_z_vals[0]
        elec_widths[i] = elec_vals[1]-elec_vals[0]
        
    return mag_y_widths, mag_z_widths, elec_widths

In [15]:
def calc_B_field(cell, electrode, elec_data):
    i_axial, d_vectors, pos_vectors = cell.get_axial_currents_from_vmem()
    i_axial = i_axial * 1e-9 #Convert to nA
    d_vectors = d_vectors * 1e-6 #Convert to um
    pos_vectors = pos_vectors * 1e-6 #Convert to um
    
    print(np.shape(i_axial))
    print(np.shape(d_vectors))
    print(np.shape(pos_vectors))
    print(np.max(i_axial))
    
    num_elecs = len(elec_data)
    time_len = np.shape(cell.tvec)[0]
    print(time_len)
    dims = 3
    all_B_fields = np.zeros((num_elecs, dims, time_len))
    
    for i in range(len(electrode.x)):
        R = np.array([electrode.x[i], electrode.y[i], electrode.z[i]])
        R = R * 1e-6 #Convert to um
        #print(R)
        B = calc_B_field_single_sensor(cell, R, i_axial, d_vectors, pos_vectors)
        all_B_fields[i] = B
        
    return all_B_fields

def calc_B_field_single_sensor(cell, R, i_axial, d_vectors, pos_vectors):
    B = np.zeros((3, cell.tvec.size))
    
    count = 0
    r_rel = R - pos_vectors
    cross = np.cross(d_vectors.T, r_rel)
    divisor = 1e7 * np.sqrt((r_rel**2).sum(axis=1))**3 
    vec = cross / np.expand_dims(divisor, axis=0).T
    
    count = 0
    for ax_curr in i_axial.T:
        B[:, count] = ax_curr @ vec
        count += 1
        
    return B

In [1]:
def get_cell_extents(cell):
    max_y = -100000
    min_y = 100000
    max_z = -100000
    min_z = 100000

    for i in range(len(cell.y3d)):
        if np.max(cell.y3d[i]) > max_y:
            max_y = np.max(cell.y3d[i])
        if np.min(cell.y3d[i]) < min_y:
            min_y = np.min(cell.y3d[i])
        if np.max(cell.z3d[i]) > max_z:
            max_z = np.max(cell.z3d[i])
        if np.min(cell.z3d[i]) < min_z:
            min_z = np.min(cell.z3d[i])

    return max_y, min_y, max_z, min_z

In [2]:
def get_apical_coords(cell, cell_name):
    ycoords = []
    zcoords = []
    
    soma_ycoords = []
    soma_zcoords = []
    
    if cell_name == 'L5_TTPC1_cADpyr232_1':
        secs = [0, 1, 3, 5, 17, 18, 30, 34, 36, 38, 40, 52, 54, 64, 66]
    else:
        raise ValueError('Apical sections for this cell have not been loaded yet')
    
    for sec in cell.allseclist:
        if 'soma' in sec.name():
            tempy = []
            tempz = []
            for n in range(sec.n3d()):
                tempy.append(sec.y3d(n))
                tempz.append(sec.z3d(n))
            soma_ycoords.append(tempy)
            soma_zcoords.append(tempz)
            
        if 'apic' in sec.name():
            tempy = []
            tempz = []
            for n in range(sec.n3d()):
                tempy.append(sec.y3d(n))
                tempz.append(sec.z3d(n))
            ycoords.append(tempy)
            zcoords.append(tempz)
            
    apic_ycoords = []
    apic_zcoords = []
    
    for i in range(len(soma_ycoords)):
        apic_ycoords.extend(soma_ycoords[i])
        apic_zcoords.extend(soma_zcoords[i])
    
    for i in secs:
        apic_ycoords.extend(ycoords[i])
        apic_zcoords.extend(zcoords[i])
        
    return apic_ycoords, apic_zcoords

In [1]:
def plotcell_1plane(cell, electrodes, lim, plot_electrodes=False, projections = [('y', 'z')]):
    fig = plt.figure(figsize = (36, 12), dpi=160)
    
    if plot_electrodes:
        elec_pairs = [(electrodes.y, electrodes.z), (electrodes.x, electrodes.y), (electrodes.x, electrodes.z)]
    
    for i in range(len(projections)):
        projection = projections[i]
        if plot_electrodes:
            X, Y = elec_pairs[i]
        
        # Plot electrodes as well as points at the "limits" of the plot to ensure entire space is plotted
        ax = fig.add_axes([0.3*i, 0, 0.25, 1], frameon=False)
        if plot_electrodes == True:
            ax.plot(X, Y, '.', marker='o', markersize=3, color='k', zorder=0)
        ax.plot([lim, lim, -lim, -lim], [lim, -lim, lim, -lim], '.', marker='o', markersize=1, color='blue')
        
        # Plot neuron itself
        pt3d_polygons = cell.get_pt3d_polygons(projection=projection)
        zips = []
        for x, y in pt3d_polygons:
            zips.append(list(zip(x, y)))
        from matplotlib.collections import PolyCollection
        polycol = PolyCollection(zips, edgecolors='none',
                                 facecolors='gray', zorder=-1, rasterized=False)
        ax.add_collection(polycol)
        ax.set_title(projection[0] + "-" + projection[1])
        ax.set_aspect('equal', 'box')
    
    return fig
